<a href="https://colab.research.google.com/github/kenanmorani/Prediction_Of_Total_Produced_Biogas_Flow/blob/main/Recursive_SVM_Standard_scaled_Feature_selective_for_Biogas_Flow_Prediction_Results.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [46]:
import pandas as pd
from sklearn import svm
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_percentage_error, accuracy_score
import math
from math import sqrt
import warnings
from sklearn import preprocessing

In [10]:
# Mounting my google drive
from google.colab import drive

In [11]:
drive.mount('/content/gdrive', force_remount=True)

Mounted at /content/gdrive


In [12]:
#Create DataFrame
ss = pd.read_csv('/content/gdrive/MyDrive/Biogaz_Flow/Real_Plant_Data.csv')

In [13]:
del ss['Date'] # Deleting the 'Date' column
ss.head()

,TMF,TKM_percent,TUKM_percent,Alcantine,Fatty_Accid,Total_Biogaz_Production
0,240,46.26,33.15,787.0,244.36,2113
1,120,44.77,22.02,983.0,413.54,1754
2,200,44.45,22.61,740.0,378.60,1884
3,160,44.31,26.64,761.0,353.66,1768
4,200,50.58,23.23,748.0,343.96,1715


In [14]:
d = preprocessing.minmax_scale(ss) # Applying normalization on the data

In [15]:
d

array([[0.07906977, 0.35562731, 0.49103836, 0.25709628, 0.13314154,
        0.07240799],
       [0.02325581, 0.34417282, 0.3261739 , 0.32112534, 0.22532065,
        0.05513592],
       [0.06046512, 0.34171279, 0.33491335, 0.24174237, 0.2062833 ,
        0.06139043],
       ...,
       [0.34418605, 0.55004613, 0.50866538, 0.14047192, 0.03456035,
        0.25316334],
       [0.47906977, 0.76737392, 0.70967264, 0.13393834, 0.02668715,
        0.22126534],
       [0.48372093, 0.66128536, 0.61161309, 0.35552463, 0.20717142,
        0.33403897]])

In [16]:
names = ss.columns
df = pd.DataFrame(d, columns=names)

In [17]:
df

,TMF,TKM_percent,TUKM_percent,Alcantine,Fatty_Accid,Total_Biogaz_Production
0,0.079070,0.355627,0.491038,0.257096,0.133142,0.072408
1,0.023256,0.344173,0.326174,0.321125,0.225321,0.055136
2,0.060465,0.341713,0.334913,0.241742,0.206283,0.061390
3,0.041860,0.340637,0.394608,0.248603,0.192695,0.055809
4,0.060465,0.388838,0.344097,0.244356,0.187409,0.053260
...,...,...,...,...,...,...
389,0.390698,0.360394,0.333284,0.100725,0.088485,0.265769
390,0.404651,0.293589,0.271515,0.090546,0.059526,0.238345
391,0.344186,0.550046,0.508665,0.140472,0.034560,0.253163
392,0.479070,0.767374,0.709673,0.133938,0.026687,0.221265


In [21]:
def Mean_Square_Error(model, x_test, y_test):
	prediction = model.predict(x_test)
	print ("Mean Square error of model:", mean_squared_error(y_test, prediction))

In [22]:
def Root_Mean_Square_Error(model, x_test, y_test):
	prediction = model.predict(x_test)
	print ("Root Mean Square error of model:", math.sqrt(mean_squared_error(y_test, prediction)))

In [23]:
def Mean_Absolute_Percentage_Error(model, x_test, y_test):
	prediction = model.predict(x_test)
	print ("Mean Absolute Percentage error of model:", mean_absolute_percentage_error(y_test, prediction))

In [44]:
import hydroeval as he
def NSE(model, x_test, y_test):
	prediction = model.predict(x_test)
	print ("NSE:", he.evaluator(he.nse, y_test, prediction))

In [47]:
def Accuracy_Score(model, x_test, y_test):
	prediction = model.predict(x_test)
	print ("Accuracy Score:", accuracy_score(y_test, prediction))

In [35]:
# Identifying the target feature by splitting the dataset
samples = df.filter(['TMF', 'TKM_percent', 'TUKM_percent', 'Alcantine', 'Fatty_Accid'])
scores = df.filter(['Total_Biogaz_Production'])

#Setting a parameter for SVM model
C = 0.1

# Defining the number of features to investigate
nFeatures = len(df.columns) - 1

rfeIndex = nFeatures

In [28]:
samples

,TMF,TKM_percent,TUKM_percent,Alcantine,Fatty_Accid
0,0.079070,0.355627,0.491038,0.257096,0.133142
1,0.023256,0.344173,0.326174,0.321125,0.225321
2,0.060465,0.341713,0.334913,0.241742,0.206283
3,0.041860,0.340637,0.394608,0.248603,0.192695
4,0.060465,0.388838,0.344097,0.244356,0.187409
...,...,...,...,...,...
389,0.390698,0.360394,0.333284,0.100725,0.088485
390,0.404651,0.293589,0.271515,0.090546,0.059526
391,0.344186,0.550046,0.508665,0.140472,0.034560
392,0.479070,0.767374,0.709673,0.133938,0.026687


In [29]:
scores

,Total_Biogaz_Production
0,0.072408
1,0.055136
2,0.061390
3,0.055809
4,0.053260
...,...
389,0.265769
390,0.238345
391,0.253163
392,0.221265


# Mean Square Error 

In [37]:
#Recursively eliminate features based on the lowest weight
while True:
	#Split into training and testing
	x_train, x_test, y_train, y_test = train_test_split(samples, scores, test_size = 0.50, train_size=0.50)
	
	#Create SVM model using a linear kernel
	model = svm.SVR(kernel='linear', C=C).fit(x_train, y_train)
	coef = model.coef_

	#Print co-efficients of features
	for i in range(0, nFeatures):
		print(samples.columns[i-1],":", coef[0][i-1])
	
	#Find the minimum weight among features and eliminate the feature with the smallest weight
	min = coef[0][0]
	index = 0
	for i in range(0, rfeIndex):
		if min > coef[0][i-1]:
			index = index + 1
			min = coef[0][i-1]
	if len(samples.columns) == 1:
		print("After recursive elimination we have the", samples.columns[index], "feature with a score of:", min)
		Mean_Square_Error(model, x_test, y_test)
		break
	else:
		print ("Lowest feature weight is for", samples.columns[index], "with a value of:", min)
		print ("Dropping feature", samples.columns[index])  

		#Drop the feature in the 'samples' dataframe based on the lowest feature index
		samples.drop(samples.columns[index], axis = 1, inplace = True)
		Mean_Square_Error(model, x_test, y_test)
		print ("\n")
		rfeIndex = rfeIndex - 1
		nFeatures = nFeatures - 1

Fatty_Accid : 0.060265365117671194
TMF : 0.42223711792726415
TKM_percent : 0.0704458258090414
TUKM_percent : 0.12843152071477865
Alcantine : 0.08337486946210618
Lowest feature weight is for TKM_percent with a value of: 0.060265365117671194
Dropping feature TKM_percent
Mean Square error of model: 0.025470455707291922


Fatty_Accid : 0.07990718535045599
TMF : 0.4906185493957137
TUKM_percent : 0.05720508092087381
Alcantine : 0.005444162071778558
Lowest feature weight is for Fatty_Accid with a value of: 0.005444162071778558
Dropping feature Fatty_Accid
Mean Square error of model: 0.020529856086165275


Alcantine : 0.026626942514316694
TMF : 0.4613953488372094
TUKM_percent : 0.12596652347800316
Lowest feature weight is for TUKM_percent with a value of: 0.026626942514316694
Dropping feature TUKM_percent
Mean Square error of model: 0.01574260862680501


Alcantine : -0.004239965241366744
TMF : 0.4727906976744186
Lowest feature weight is for Alcantine with a value of: -0.004239965241366744
Drop

/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)
/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)
/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)
/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return

# Root Mean Square Error

In [38]:
# Identifying the target feature by splitting the dataset
samples = df.filter(['TMF', 'TKM_percent', 'TUKM_percent', 'Alcantine', 'Fatty_Accid'])
scores = df.filter(['Total_Biogaz_Production'])

#Setting a parameter for SVM model
C = 0.1

# Defining the number of features to investigate
nFeatures = len(df.columns) - 1

rfeIndex = nFeatures

In [39]:
#Recursively eliminate features based on the lowest weight
while True:
	#Split into training and testing
	x_train, x_test, y_train, y_test = train_test_split(samples, scores, test_size = 0.50, train_size=0.5)
	
	#Create SVM model using a linear kernel
	model = svm.SVR(kernel='linear', C=C).fit(x_train, y_train)
	coef = model.coef_

	#Print co-efficients of features
	for i in range(0, nFeatures):
		print(samples.columns[i-1],":", coef[0][i-1])
	
	#Find the minimum weight among features and eliminate the feature with the smallest weight
	min = coef[0][0]
	index = 0
	for i in range(0, rfeIndex):
		if min > coef[0][i-1]:
			index = index + 1
			min = coef[0][i-1]
	if len(samples.columns) == 1:
		print("After recursive elimination we have the", samples.columns[index], "feature with a score of:", min)
		Root_Mean_Square_Error(model, x_test, y_test)
		break
	else:
		print ("Lowest feature weight is for", samples.columns[index], "with a value of:", min)
		print ("Dropping feature", samples.columns[index])  

		#Drop the feature in the 'samples' dataframe based on the lowest feature index
		samples.drop(samples.columns[index], axis = 1, inplace = True)
		Root_Mean_Square_Error(model, x_test, y_test)
		print ("\n")
		rfeIndex = rfeIndex - 1
		nFeatures = nFeatures - 1

Fatty_Accid : 0.034870923098717393
TMF : 0.45906976744186057
TKM_percent : 0.07210947109471096
TUKM_percent : 0.14922233743149163
Alcantine : -0.058315120985524875
Lowest feature weight is for TUKM_percent with a value of: -0.058315120985524875
Dropping feature TUKM_percent
Root Mean Square error of model: 0.1479612577153315


Fatty_Accid : 0.08834346566813592
TMF : 0.46054320229536316
TKM_percent : 0.07365196233638827
Alcantine : 4.685882842340311e-05
Lowest feature weight is for Fatty_Accid with a value of: 4.685882842340311e-05
Dropping feature Fatty_Accid
Root Mean Square error of model: 0.14060904669839455


Alcantine : 0.006593545000335522
TMF : 0.4921811101404076
TKM_percent : 0.08949931887178011
Lowest feature weight is for TKM_percent with a value of: 0.006593545000335522
Dropping feature TKM_percent
Root Mean Square error of model: 0.14173575802559235


Alcantine : -0.02450182329512647
TMF : 0.4685326595118863
Lowest feature weight is for Alcantine with a value of: -0.0245018

/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)
/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)
/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)
/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return

# Mean Absolute Percentage Error

In [40]:
# Identifying the target feature by splitting the dataset
samples = df.filter(['TMF', 'TKM_percent', 'TUKM_percent', 'Alcantine', 'Fatty_Accid'])
scores = df.filter(['Total_Biogaz_Production'])

#Setting a parameter for SVM model
C = 0.1

# Defining the number of features to investigate
nFeatures = len(df.columns) - 1

rfeIndex = nFeatures

In [41]:
#Recursively eliminate features based on the lowest weight
while True:
	#Split into training and testing
	x_train, x_test, y_train, y_test = train_test_split(samples, scores, test_size = 0.50, train_size=0.5)
	
	#Create SVM model using a linear kernel
	model = svm.SVR(kernel='linear', C=C).fit(x_train, y_train)
	coef = model.coef_

	#Print co-efficients of features
	for i in range(0, nFeatures):
		print(samples.columns[i-1],":", coef[0][i-1])
	
	#Find the minimum weight among features and eliminate the feature with the smallest weight
	min = coef[0][0]
	index = 0
	for i in range(0, rfeIndex):
		if min > coef[0][i-1]:
			index = index + 1
			min = coef[0][i-1]
	if len(samples.columns) == 1:
		print("After recursive elimination we have the", samples.columns[index], "feature with a score of:", min)
		Mean_Absolute_Percentage_Error(model, x_test, y_test)
		break
	else:
		print ("Lowest feature weight is for", samples.columns[index], "with a value of:", min)
		print ("Dropping feature", samples.columns[index])  

		#Drop the feature in the 'samples' dataframe based on the lowest feature index
		samples.drop(samples.columns[index], axis = 1, inplace = True)
		Mean_Absolute_Percentage_Error(model, x_test, y_test)
		print ("\n")
		rfeIndex = rfeIndex - 1
		nFeatures = nFeatures - 1

Fatty_Accid : 0.12402068674514577
TMF : 0.45773691418522267
TKM_percent : 0.08176853964403846
TUKM_percent : 0.06157214479947989
Alcantine : -0.01792427078394075
Lowest feature weight is for Fatty_Accid with a value of: -0.01792427078394075
Dropping feature Fatty_Accid
Mean Absolute Percentage error of model: 0.7346963248363217


Alcantine : 0.06365281946554778
TMF : 0.4708054414840145
TKM_percent : 0.006847213442698924
TUKM_percent : 0.04523464071943977
Lowest feature weight is for TUKM_percent with a value of: 0.006847213442698924
Dropping feature TUKM_percent
Mean Absolute Percentage error of model: 0.481619171687182


Alcantine : 0.01531062455777692
TMF : 0.44844354552187454
TKM_percent : 0.0762504270254013
Lowest feature weight is for TKM_percent with a value of: 0.01531062455777692
Dropping feature TKM_percent
Mean Absolute Percentage error of model: 3053556247443.168


Alcantine : 0.026687985017923854
TMF : 0.479427333416226
Lowest feature weight is for Alcantine with a value of

/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)
/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)
/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)
/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return

# Nash-Sutcliffe Efficiency 

In [42]:
# Identifying the target feature by splitting the dataset
samples = df.filter(['TMF', 'TKM_percent', 'TUKM_percent', 'Alcantine', 'Fatty_Accid'])
scores = df.filter(['Total_Biogaz_Production'])

#Setting a parameter for SVM model
C = 0.1

# Defining the number of features to investigate
nFeatures = len(df.columns) - 1

rfeIndex = nFeatures

In [45]:
#Recursively eliminate features based on the lowest weight
while True:
	#Split into training and testing
	x_train, x_test, y_train, y_test = train_test_split(samples, scores, test_size = 0.50, train_size=0.5)
	
	#Create SVM model using a linear kernel
	model = svm.SVR(kernel='linear', C=C).fit(x_train, y_train)
	coef = model.coef_

	#Print co-efficients of features
	for i in range(0, nFeatures):
		print(samples.columns[i-1],":", coef[0][i-1])
	
	#Find the minimum weight among features and eliminate the feature with the smallest weight
	min = coef[0][0]
	index = 0
	for i in range(0, rfeIndex):
		if min > coef[0][i-1]:
			index = index + 1
			min = coef[0][i-1]
	if len(samples.columns) == 1:
		print("After recursive elimination we have the", samples.columns[index], "feature with a score of:", min)
		NSE(model, x_test, y_test)
		break
	else:
		print ("Lowest feature weight is for", samples.columns[index], "with a value of:", min)
		print ("Dropping feature", samples.columns[index])  

		#Drop the feature in the 'samples' dataframe based on the lowest feature index
		samples.drop(samples.columns[index], axis = 1, inplace = True)
		NSE(model, x_test, y_test)
		print ("\n")
		rfeIndex = rfeIndex - 1
		nFeatures = nFeatures - 1

Fatty_Accid : 0.09782099178070802
TMF : 0.4564922771873922
TKM_percent : 0.12049724659022157
TUKM_percent : 0.11210409777620711
Fatty_Accid : 0.09782099178070802
Lowest feature weight is for TKM_percent with a value of: 0.09782099178070802
Dropping feature TKM_percent
NSE: [-1.20125781]


Fatty_Accid : 0.08308843407688868
TMF : 0.4993231737804894
TUKM_percent : 0.04515690986712856
Fatty_Accid : 0.08308843407688868
Lowest feature weight is for Fatty_Accid with a value of: 0.04515690986712856
Dropping feature Fatty_Accid
NSE: [-0.75443839]


TUKM_percent : 0.08644342440329285
TMF : 0.45032443680110235
TUKM_percent : 0.08644342440329285
Lowest feature weight is for TUKM_percent with a value of: 0.08644342440329285
Dropping feature TUKM_percent
NSE: [-1.27311901]


TMF : 0.4379069767441861
TMF : 0.4379069767441861
After recursive elimination we have the TMF feature with a score of: 0.4379069767441861
NSE: [-1.74880533]


/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)
/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)
/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)
/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return